In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris


# Import PySwarms
import pyswarms as ps

# Some more magic so that the notebook will reload external python modules;
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [89]:
f= open("two_spirals.dat","r")
data = np.loadtxt(f)

In [134]:
# Store the features as X and the labels as y
X = data[:,0:2]
y = data[:,2]
y=y.astype(int)
print(len(X))
print(len(y))
print(y)

264
264
[1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0 1
 0 1 0 1 0]


In [135]:
# data = load_iris()
# X=data.data
# y=data.target
# print(len(x))

In [136]:
n_inputs = 2
n_hidden = 8
n_classes = 2

parameters = (n_inputs*n_hidden)+(n_hidden*n_classes)+n_hidden+n_classes
num_samples = len(X)

print(num_samples)
print(parameters)

264
42


In [137]:
def logits_function(p):
    """ Calculate roll-back the weights and biases

    Inputs
    ------
    p: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    numpy.ndarray of logits for layer 2

    """
    # Roll-back the weights and biases
    W1 = p[0:n_inputs*n_hidden].reshape((n_inputs,n_hidden))
    b1 = p[n_inputs*n_hidden:n_inputs*n_hidden+n_hidden].reshape((n_hidden,))
    W2 = p[n_inputs*n_hidden+n_hidden:n_inputs*n_hidden+n_hidden+n_hidden*n_classes].reshape((n_hidden,n_classes))
    b2 = p[n_inputs*n_hidden+n_hidden+n_hidden*n_classes:n_inputs*n_hidden+n_hidden+n_hidden*n_classes+n_classes].reshape((n_classes,))

    # Perform forward propagation
    z1 = X.dot(W1) + b1  # Pre-activation in Layer 1
    a1 = np.tanh(z1)     # Activation in Layer 1
    logits = a1.dot(W2) + b2 # Pre-activation in Layer 2
    return logits          # Logits for Layer 2

In [141]:
def forward_prop(params):
    """Forward propagation as objective function

    This computes for the forward propagation of the neural network, as
    well as the loss.

    Inputs
    ------
    params: np.ndarray
        The dimensions should include an unrolled version of the
        weights and biases.

    Returns
    -------
    float
        The computed negative log-likelihood loss given the parameters
    """


    logits = logits_function(params)

    # Compute for the softmax of the logits
    exp_scores = np.exp(logits)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
#     print("probs = ",len(probs))
#     print("probs ==",probs[0])

#     print("probs ==",probs[0,0])
    # Compute for the negative log likelihood
    
#     print(range(num_samples),len(y))
#     test = np.zeros(y.size)
#     print(test.size)
#     for i in range(test.size):
#         print(y[i])
#         test[i] = probs[i][y[0]]
#         print(test[i])
    corect_logprobs = -np.log(probs[range(num_samples), y])
    loss = np.sum(corect_logprobs) / num_samples

   
    return loss

In [142]:

def f(x):
    """Higher-level method to do forward_prop in the
    whole swarm.

    Inputs
    ------
    x: numpy.ndarray of shape (n_particles, dimensions)
        The swarm that will perform the search

    Returns
    -------
    numpy.ndarray of shape (n_particles, )
        The computed loss for each particle
    """
    n_particles = x.shape[0]
    j = [forward_prop(x[i]) for i in range(n_particles)]
    return np.array(j)

In [143]:
%%time
# Initialize swarm
options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}

# Call instance of PSO
dimensions = (n_inputs * n_hidden) + (n_hidden * n_classes) + n_hidden + n_classes
optimizer = ps.single.GlobalBestPSO(n_particles=100, dimensions=dimensions, options=options)

# Perform optimization
cost, pos = optimizer.optimize(f, iters=1000)


2020-11-10 13:03:56,355 - pyswarms.single.global_best - INFO - Optimize for 1000 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best: 100%|██████████████████████████████████████████████████████████|1000/1000, best_cost=0.609
2020-11-10 13:04:07,492 - pyswarms.single.global_best - INFO - Optimization finished | best cost: 0.6087138652442746, best pos: [ 1.75107196e+00  2.37738952e-01  3.31570160e+00  2.88853120e+00
  5.38590521e-01  1.09581084e+00  3.63463833e-01  1.55889801e+00
  3.31143578e+00  1.49423055e+00  1.36370859e+00  2.14405116e+00
  4.13687665e+02  6.99035336e-01  8.30861727e-01  1.80018041e+00
  8.95733483e-01  9.99242160e-01  1.90811653e+00  1.89970856e+00
  1.12891134e+00  1.00447711e+00 -1.17539089e+00  2.59068959e+00
  1.14355409e+00  1.25441919e-01  5.42180175e-01  2.23963328e+00
  1.15355370e+00  3.27339010e-01  3.15287343e+00  1.71673163e+00
  2.03504540e+00  6.40628840e-01  1.80610137e-01  1.65744968e+00
  1.30567939e-01  1.35420128e+00  8.050873

Wall time: 11.1 s


In [146]:
def predict(pos):
    """
    Use the trained weights to perform class predictions.

    Inputs
    ------
    pos: numpy.ndarray
        Position matrix found by the swarm. Will be rolled
        into weights and biases.
    """
    logits = logits_function(pos)
    y_pred = np.argmax(logits, axis=1)
    return y_pred

In [147]:
(predict(pos) == y).mean()

0.6439393939393939